In [15]:
## analysis homoeologous gene position in TAD
import pandas as pd
## df1 include gene, df2 include TAD
def region_overlap(df1, df2):
    df_return = pd.DataFrame(columns=['c', 's', 'e', 'gid'])
    i = 0
    for index, row in df1.iterrows():
        c1, s1, e1, gid = row['c'], row['s'], row['e'], row['gid']
        df_tmp = pd.DataFrame(df2.query("c==@c1 & s<=@e1 & e>=@s1"))
        if df_tmp.shape[0] >0:
            print(df_tmp)
            df_return.loc[i, :] = [c1, s1, e1, gid]
            i+=1
    #print(df_return) 
    df_return['tags'] = ['boundary']*df_return.shape[0]
    return df_return

In [16]:
## find boundary of TAD
df_tad = pd.read_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/TM-1_TADLib_tad_20K_2M_choosed.bed', sep='\t', names=['c', 's', 'e', 'o'])
df_boundary = df_tad[['c', 's']].copy()
df_boundary.columns = ['c', 'center']
df_boundary['s'] = df_boundary['center'] - 100000
df_boundary['e'] = df_boundary['center'] + 100000

In [17]:
### get gene postion 
chrs = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13',
        'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
#df_gene = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/all_homo_gene_position.txt', sep='\t', names=['c', 's', 'e', 'gid'])
df_gene = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/test.txt', sep='\t', names=['c', 's', 'e', 'gid'])
df_gene_copy = df_gene.copy()
for c in chrs:
    df_gene_chr = df_gene.query('c == @c')
    df_boundary_chr = df_boundary.query('c == @c')
    df_tmp = region_overlap(df_gene_chr, df_boundary_chr)
    df_gene_copy = df_gene_copy.append(df_tmp, ignore_index=True)
df_gene_copy.fillna('noboundary', inplace=True)
print(df_gene_copy)

             c    center         s         e
5818  Ghir_D11  60780000  60680000  60880000
          c         s         e              gid        tags
0  Ghir_D11  60692682  60696961  Ghir_D11G028500  noboundary
1  Ghir_D11  60692682  60696961  Ghir_D11G028500    boundary


In [28]:
### number of gene licated in boundary
print(df_gene_copy.query("tags=='boundary'").shape[0])

38085


In [42]:
### homo bias gene linked with TAD position
df_bias = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/bias/all_bias_expression.txt', sep='\t')
print(df_bias.shape[0])
df_boundary_gid = df_gene_copy.query("tags=='boundary'")[['gid', 'tags']]
df_boundary_gid.columns = ['Geneid_at', 'tags_at']
df_bias=pd.merge(df_bias, df_boundary_gid, how='left', on=['Geneid_at'])
df_boundary_gid.columns = ['Geneid_dt', 'tags_dt']
df_bias=pd.merge(df_bias, df_boundary_gid, how='left', on=['Geneid_dt'])
df_bias.fillna('noboundary', inplace=True)

2886


In [46]:
print("bias表达的同源基因对与TAD上的分布相关的有: {}".format(df_bias.query('tags_at!=tags_dt').shape[0]))
df_out = df_bias.query('tags_at!=tags_dt')
df_out.to_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/bias/TAD/bias_gene_TAD_diff.txt', sep='\t', index=False)

bias表达的同源基因对与TAD上的分布相关的有: 857


In [49]:
## 分析TAD上分布差异的同源基因对与bias的直接联系
df_bias_tad = df_out.query("(tags_at=='boundary' & tags_dt=='noboundary' & Bias=='biasA') | (tags_at=='noboundary' & tags_dt=='boundary' & Bias=='biasD')")
df_bias_tad.to_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/bias/TAD/bias_gene_TAD_True.txt', sep='\t', index=False)
print('bias表达的同源基因对与TAD上的分布直接相关的有: {}'.format(df_bias_tad.shape[0]))

bias表达的同源基因对与TAD上的分布直接相关的有: 421


In [58]:
#根据基因ID查找其所位于的TAD
geneid = 'Ghir_A11G034830'
gene_position = pd.DataFrame(df_gene.query("gid==@geneid"))
gene_position.reset_index(drop=True, inplace=True)
c, s, e = gene_position.loc[0, 'c'], gene_position.loc[0, 's'], gene_position.loc[0, 'e']
print(c, s, e)

Ghir_A11 122602735 122604937


In [61]:
## 分析A亚组TAD内所包含的同源基因对
df_gene_pairs  = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/all_homo_gene_pairs_position.txt', sep='\t')
c, s,e = 'Ghir_A03', 19480000, 19860000
df_gene_region = df_gene_pairs.query("chr_at == @c & start_at<=@e & end_at>=@s")
print(df_gene_region[['geneid_at', 'chr_at', 'start_at', 'end_dt', 'geneid_dt', 'chr_dt', 'start_dt', 'end_dt']])

            geneid_at    chr_at  start_at    end_dt        geneid_dt  \
3685  Ghir_A03G008200  Ghir_A03  19474979  36973100  Ghir_D03G010660   
3686  Ghir_A03G008210  Ghir_A03  19512154  37003724  Ghir_D03G010670   
3687  Ghir_A03G008230  Ghir_A03  19541216  37022724  Ghir_D03G010680   
3688  Ghir_A03G008250  Ghir_A03  19634972  37060583  Ghir_D03G010690   
3689  Ghir_A03G008260  Ghir_A03  19665524  37075205  Ghir_D03G010700   
3690  Ghir_A03G008270  Ghir_A03  19670966  37079436  Ghir_D03G010710   
3691  Ghir_A03G008280  Ghir_A03  19688626  37105694  Ghir_D03G010730   
3692  Ghir_A03G008290  Ghir_A03  19699235  37114674  Ghir_D03G010740   
3693  Ghir_A03G008300  Ghir_A03  19758220  37158674  Ghir_D03G010760   
3694  Ghir_A03G008310  Ghir_A03  19762589  37170506  Ghir_D03G010780   
3695  Ghir_A03G008320  Ghir_A03  19841797  37241209  Ghir_D03G010790   
3696  Ghir_A03G008330  Ghir_A03  19851844  37250389  Ghir_D03G010800   

        chr_dt  start_dt    end_dt  
3685  Ghir_D03  36963891  

In [8]:
## 分析bias基因同时满足TAD位置 差异的同源基因对在chip修饰水平上的差异
import pandas as pd
df         = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/bias/TAD/bias_gene_TAD_True.txt', sep='\t')
df_h3k4me3 = pd.read_csv('F:/TM-1_leaf_3D-genome/ChIP/peaks/TM1_H3K4me3_Rep3_peaks.narrowPeak', sep='\t', names=['chr', 'start', 'end'])
df_h3k27ac = pd.read_csv('F:/TM-1_leaf_3D-genome/ChIP/peaks/TM1_H3K27ac_Rep1_peaks.narrowPeak', sep='\t', names=['chr', 'start', 'end'])
# df_h3k27ac = pd.read_csv('F:/TM-1_leaf_3D-genome/ChIP/peaks/TM1_H3K9ac_Rep1_peaks.narrowPeak', sep='\t', names=['chr', 'start', 'end'])
## h3k4me3
for index, row in df.iterrows(): 
    at_c, at_s, at_e = row['Chr_at'], int(row['Start_at'])-3000, int(row['End_at'])+500
    dt_c, dt_s, dt_e = row['Chr_dt'], int(row['Start_dt'])-3000, int(row['End_dt'])+500
    df_at_tmp = df_h3k4me3.query("chr==@at_c & start <= @at_e & end >= @at_s")
    df_dt_tmp = df_h3k4me3.query("chr==@dt_c & start <= @dt_e & end >= @dt_s")
    if df_at_tmp.shape[0]: 
        df.loc[index, 'at_k4me3'] = 'Y'
    else: 
        df.loc[index, 'at_k4me3'] = 'N'
    if df_dt_tmp.shape[0]: 
        df.loc[index, 'dt_k4me3'] = 'Y'
    else: 
        df.loc[index, 'dt_k4me3'] = 'N'
## h3k27ac
for index, row in df.iterrows(): 
    at_c, at_s, at_e = row['Chr_at'], int(row['Start_at'])-3000, int(row['End_at'])+500
    dt_c, dt_s, dt_e = row['Chr_dt'], int(row['Start_dt'])-3000, int(row['End_dt'])+500
    df_at_tmp = df_h3k27ac.query("chr==@at_c & start <= @at_e & end >= @at_s")
    df_dt_tmp = df_h3k27ac.query("chr==@dt_c & start <= @dt_e & end >= @dt_s")
    if df_at_tmp.shape[0]: 
        df.loc[index, 'at_k27ac'] = 'Y'
    else: 
        df.loc[index, 'at_k27ac'] = 'N'
    if df_dt_tmp.shape[0]: 
        df.loc[index, 'dt_k27ac'] = 'Y'
    else: 
        df.loc[index, 'dt_k27ac'] = 'N'
## h3k9me2
df.to_csv("F:/TM-1_leaf_3D-genome/RNA_Seq/bias/TAD/bias_gene_TAD_chip.txt", sep='\t', index=False)

In [3]:
## 选出符合要求的同源基因对，例如：biasA 同时 at 亚组有K4me3的peaks 而dt 亚组没有
import pandas as pd
df = pd.read_csv("F:/TM-1_leaf_3D-genome/RNA_Seq/bias/TAD/bias_gene_TAD_chip.txt", sep='\t')
df_need = df.query("(Bias=='biasA' & at_k4me3=='Y' & at_k27ac=='Y' & dt_k4me3=='N' & dt_k27ac=='N') | (Bias=='biasD' & dt_k4me3=='Y' & dt_k27ac=='Y' & at_k4me3=='N' & at_k27ac=='N')")
df_need.to_csv("F:/TM-1_leaf_3D-genome/RNA_Seq/bias/TAD/bias_gene_TAD_chip_need.txt", sep='\t', index=False)
print("有{}同源基因对满足bias同时包含对应的chip peaks".format(df_need.shape[0]))

有43同源基因对满足bias同时包含对应的chip peaks


In [10]:
## 对这满足条件的43对同源基因生成作图文件
fp = open("F:/TM-1_leaf_3D-genome/RNA_Seq/bias/TAD/bias_gene_TAD_plot.txt", 'w')
for index, row in df_need.iterrows():
    at_gid, at_chr, at_s, at_e, dt_chr, dt_s, dt_e = row['Geneid_at'], row['Chr_at'], row['Start_at']-200000, row['End_at']+200000, row['Chr_dt'], row['Start_dt']-200000, row['End_dt']+200000
    #line_at = "bsub -q high -e %J.err -o %J.out -R span[hosts=1] -J plot 'pyGenomeTracks --tracks loop.ini --region {0}:{1}-{2} -o TAD_figure/{3}_at.pdf'".format(at_chr, at_s, at_e, at_gid)
    #line_dt = "bsub -q high -e %J.err -o %J.out -R span[hosts=1] -J plot 'pyGenomeTracks --tracks loop.ini --region {0}:{1}-{2} -o TAD_figure/{3}_dt.pdf'".format(dt_chr, dt_s, dt_e, at_gid)
    line_at = "sh plot_loop.sh {0} {1} {2}".format(at_chr, at_s, at_e)
    line_dt = "sh plot_loop.sh {0} {1} {2}".format(dt_chr, dt_s, dt_e)
    print(line_at, file=fp)
    print(line_dt, file=fp)
fp.close()

In [14]:
## 生成同源基因位置文件
df_out = pd.DataFrame(columns=['c1', 's1', 'e1', 'c2', 's2', 'e2', 'one', 'two', 'three', 'four', 'color'])
for index, row in df_need.iterrows():
    at_chr, at_s, at_e, dt_chr, dt_s, dt_e =  row['Chr_at'], row['Start_at']-20000, row['End_at']+20000, row['Chr_dt'], row['Start_dt']-20000, row['End_dt']+20000
    df_out.loc[df_out.shape[0], :] = at_chr, at_s, at_e, at_chr, at_s, at_e, '.', '.', '.', '.', '255,0,255'
    df_out.loc[df_out.shape[0], :] = dt_chr, dt_s, dt_e, dt_chr, dt_s, dt_e, '.', '.', '.', '.', '255,0,255'
df_out.to_csv("F:/TM-1_leaf_3D-genome/RNA_Seq/bias/TAD/bias_gene_TAD_position.bedpe", sep='\t', index=False, header=False)


In [65]:
## 随机选择9115对同源基因，并统计其中具有bias表达的基因数目
import pandas as pd
from numpy import *
df_homo_gene = pd.read_csv("G:/TM-1_leaf_3D-genome/RNA_Seq/all_homo_gene_pairs_position.txt", sep="\t")
df_bias_gene = pd.read_csv("G:/TM-1_leaf_3D-genome/RNA_Seq/bias/all_bias_expression.txt", sep="\t")
df_bias_gene.rename(columns = {'Geneid_at':'geneid_at', 'Geneid_dt':'geneid_dt'}, inplace=True)
df_TAD_gene = pd.read_csv("G:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/gene_position/hg_TAD_position.txt", sep="\t")
df_diff_TAD_gene = df_TAD_gene.query("tags_at != tags_dt").copy()
sums = []
for i in range(100):
    df_sample_gene = df_homo_gene.sample(n=9115, replace=False, axis=0)
    df_sample_bias_gene = pd.merge(df_sample_gene, df_bias_gene, how='inner', on=['geneid_at', 'geneid_dt'])
    df_diff_TAD_gene_bias = pd.merge(df_sample_bias_gene, df_diff_TAD_gene, how='inner', on=['geneid_at', 'geneid_dt'])
    sums.append(df_diff_TAD_gene_bias.shape[0])
print(sums)
print(mean(sums))



[422, 435, 457, 442, 430, 426, 423, 465, 432, 436, 426, 432, 439, 418, 425, 405, 445, 430, 418, 431, 407, 424, 434, 410, 432, 425, 433, 429, 434, 399, 451, 411, 421, 428, 411, 403, 421, 411, 457, 428, 418, 425, 444, 442, 434, 415, 398, 430, 410, 434, 390, 429, 445, 411, 452, 391, 418, 417, 433, 413, 419, 409, 399, 429, 438, 404, 426, 396, 435, 441, 402, 435, 442, 402, 416, 407, 423, 447, 431, 403, 398, 452, 382, 443, 425, 413, 447, 397, 408, 430, 412, 404, 430, 423, 458, 442, 436, 413, 423, 435]
424.3
